In [1]:
using Revise

In [2]:
using Pkg

In [3]:
Pkg.activate(".")

  Activating environment at `~/Documents/julia/TowerOfHanoi/Project.toml`


In [4]:
Pkg.project()

Pkg.Types.ProjectInfo("TowerOfHanoi", UUID("6a97d50a-75bd-41c1-a5a5-4807027c1e39"), v"0.1.0", true, Dict{String, Base.UUID}(), "/home/btbuxton/Documents/julia/TowerOfHanoi/Project.toml")

In [18]:
Pkg.test()

     Testing TowerOfHanoi
      Status `/tmp/jl_QE198A/Project.toml`
  [6a97d50a] TowerOfHanoi v0.1.0 `~/Documents/julia/TowerOfHanoi`
  [8dfed614] Test `@stdlib/Test`
      Status `/tmp/jl_QE198A/Manifest.toml`
  [6a97d50a] TowerOfHanoi v0.1.0 `~/Documents/julia/TowerOfHanoi`
  [2a0f44e3] Base64 `@stdlib/Base64`
  [b77e0a4c] InteractiveUtils `@stdlib/InteractiveUtils`
  [56ddb016] Logging `@stdlib/Logging`
  [d6f4376e] Markdown `@stdlib/Markdown`
  [9a3f8284] Random `@stdlib/Random`
  [9e88b42a] Serialization `@stdlib/Serialization`
  [8dfed614] Test `@stdlib/Test`
     Testing Running tests...


Test Summary: | Pass  Total
Domain        |   14     14


     Testing TowerOfHanoi tests passed 


In [83]:
using TowerOfHanoi

In [6]:
Tower(3)

   ***                     
  *****                    
 *******                   


In [544]:
module Data

struct Disc
    value::Integer
end

Base.show(io::IO, disc::Disc) = print(io, repeat("*", disc.value))

Base.:(<)(a::Disc, b::Disc) = a.value < b.value
Base.:(==)(a::Disc, b::Disc) = a.value == b.value
Base.hash(a::Disc,h::UInt) = hash(a.value,h)

function show_line(disc::Disc, max::Integer, depth::Integer)
    rpad(lpad(repeat("*", disc.value), Integer((max - disc.value) / 2 + disc.value)), max)
end

struct Rod
    discs::AbstractVector{Disc}
end

Rod() = Rod([])
Base.:(==)(a::Rod,b::Rod) = a.discs == b.discs
Base.hash(a::Rod,h::UInt) = hash(a.discs,h)

function show_line(rod::Rod, max::Integer, depth::Integer)
    disc = get(rod.discs, depth, nothing)
    if disc == nothing
        repeat(" ", max)
    else    
        show_line(disc, max, depth)
    end
end

top_disc(rod::Rod) = last(rod.discs)
has_discs(rod::Rod) = !isempty(rod.discs)

function can_accept(rod::Rod, disc::Disc)
    if !has_discs(rod)
        return true
    end
    top_disc(rod) > disc
end

struct Tower
    size::Integer
    rods::AbstractVector{Rod}
end

function Tower(size::Integer)
    stack = reverse([Disc(each) for each in range(3, step=2,length=size)])
    Tower(size, [Rod(stack), Rod(), Rod()])
end

Base.:(==)(a::Tower,b::Tower) = a.size == b.size && a.rods == b.rods
Base.hash(a::Tower,h::UInt) = hash(a.rods,h)

function can_make_move(tower::Tower, from::Integer, to::Integer)
    if from == to
        return false, "Can not be same rod[$from]"
    end
    
    from_rod = tower.rods[from]
    if !has_discs(from_rod)
        return false, "No discs to move from rod[$from]"
    end
    
    if !can_accept(tower.rods[to], top_disc(from_rod))
        return false, "Disc at top of rod[$from] is greater than top of rod[$to]"
    end
    true, nothing
end

function Base.show(io::IO, tower::Tower)
    max = tower.size * 2 + 3
    for y in range(tower.size, length=tower.size, step=-1)
        for each_rod in tower.rods
            print(io, show_line(each_rod, max, y))
        end
        println(io)
    end
end

struct InvalidMoveException <: Exception
    value::String
end

abstract type MoveState end
struct Valid <: MoveState end
struct Invalid <: MoveState end
struct Possible <: MoveState end

struct Move{T}
    tower::Tower
    from::Integer
    to::Integer
    msg::String # less than ideal - only for invalid
    
    Move{Valid}(move::Move{Possible}) = new{Valid}(move.tower, move.from, move.to, "") # yuck
    Move{Invalid}(move::Move{Possible}, msg::String) = new{Invalid}(move.tower,move.from,move.to,msg)
    Move{Possible}(tower::Tower, from::Integer, to::Integer) = new{Possible}(tower,from,to,"")
    Move(tower::Tower, from::Integer, to::Integer) = Move{Possible}(tower,from,to)
end

function (move::Move{Possible})() 
    check(move)()
end

function (move::Move{Invalid})()
    throw(InvalidMoveException(move.msg))
end

function (move::Move{Valid})()
    rods = deepcopy(move.tower.rods)
    from_rod = rods[move.from]
    to_rod = rods[move.to]
    
    disc = pop!(from_rod.discs)
    push!(to_rod.discs, disc)
    Tower(move.tower.size, rods)
end

function check(move::Move{Possible})
    is_valid, msg = can_make_move(move.tower,move.from,move.to)
    if is_valid
        Move{Valid}(move)
    else
        Move{Invalid}(move, msg)
    end
end

function add_if_valid(move::Move{Possible}, all::AbstractVector{Any})
    add_if_valid(check(move), all)
end

add_if_valid(move::Move{Valid}, all::AbstractVector{Any}) = push!(all, move)
add_if_valid(move::Move{Invalid}, ::AbstractVector{Any}) = nothing

function valid_moves(tower::Tower)
    result = []
    for from_index in range(1, length=3)
        for to_index in range(1, length=3)
            if from_index === to_index
                continue
            end
            move = Move(tower, from_index, to_index)
            add_if_valid(move, result)
        end
    end
    return result
end


end

#TODO Move is wasteful - an experiment - better, but still has issues
#TODO save this and start below
#TODO tests

Main.Data

In [545]:
#Tower() = Tower([Rod(), Rod(), Rod()])

In [546]:
t = Data.Tower(4)

    ***                          
   *****                         
  *******                        
 *********                       


In [547]:
Data.valid_moves(t)

2-element Vector{Any}:
 Main.Data.Move{Main.Data.Valid}(    ***                          
   *****                         
  *******                        
 *********                       
, 1, 2, "")
 Main.Data.Move{Main.Data.Valid}(    ***                          
   *****                         
  *******                        
 *********                       
, 1, 3, "")

In [548]:
[hash(each) for each in Data.valid_moves(t)]

2-element Vector{UInt64}:
 0x55346322cc8ee4fb
 0xdf079c7d5e3049c9

In [534]:
Data.can_accept(t.rods[1], Data.Disc(7))

false

In [535]:
Data.Move(t,1,2)()

                                 
   *****                         
  *******                        
 *********     ***               


In [536]:
Data.Move(t,2,1)()

LoadError: Main.Data.InvalidMoveException("No discs to move from rod[2]")

In [537]:
Data.Move(t,1,2)

Main.Data.Move{Main.Data.Possible}(    ***                          
   *****                         
  *******                        
 *********                       
, 1, 2, "")

In [538]:
r = Data.Move(t,1,2)()
r = Data.Move(r,1,3)()
r = Data.Move(r,2,3)()

                                 
                                 
  *******                 ***    
 *********               *****   


In [539]:
m = Data.Move(r,1,3)
Data.check(m)

Main.Data.Move{Main.Data.Invalid}(                                 
                                 
  *******                 ***    
 *********               *****   
, 1, 3, "Disc at top of rod[1] is greater than top of rod[3]")

In [540]:
Data.can_accept(r.rods[3], Data.top_disc(r.rods[1]))

false

In [541]:
println(hash(t))
t

3994254288365766802


    ***                          
   *****                         
  *******                        
 *********                       


In [542]:
r = Data.Move(r,3,2)()
r = Data.Move(r,3,1)()
r = Data.Move(r,2,1)()

    ***                          
   *****                         
  *******                        
 *********                       


In [543]:
println(hash(r))
r

3994254288365766802


    ***                          
   *****                         
  *******                        
 *********                       


In [76]:
println(t.rods[1].discs[1])
println(t.rods[1].discs[2])

*********
*******


In [82]:
Data.display(t.rods[1], 11, 3)

"   *****   "

In [80]:
Data.display(Data.Disc(3), 5, 1)

" *** "

In [19]:
[Data.Disc(each) for each in range(3, step=2, length=UInt8(8))]

8-element Vector{Main.Data.Disc}:
 ***
 *****
 *******
 *********
 ***********
 *************
 ***************
 *****************

In [28]:
string(Data.Disc(7))

"*******"

In [30]:
rpad(lpad("***", 5, '.'), 7, '.')

"..***.."

In [31]:
length("***")

3

In [53]:
Data.display(Data.Disc(3), 9)

"...***..."

In [50]:
(11 - 3) / 2 + 3

7.0

In [9]:
a = [0,1]

2-element Vector{Int64}:
 0
 1

In [15]:
push!(a,3)

4-element Vector{Int64}:
 0
 1
 3
 3

In [16]:
a

4-element Vector{Int64}:
 0
 1
 3
 3

In [18]:
b=copy(a)

4-element Vector{Int64}:
 0
 1
 3
 3

In [19]:
pop!(b)

3

In [20]:
b

3-element Vector{Int64}:
 0
 1
 3

In [22]:
copy(t)

LoadError: MethodError: no method matching copy(::Tower)
[0mClosest candidates are:
[0m  copy([91m::MbedTLS.MD[39m) at /home/btbuxton/.julia/packages/MbedTLS/4YY6E/src/md.jl:92
[0m  copy([91m::LinearAlgebra.Adjoint{var"#s814", LinearAlgebra.Rotation{T}} where var"#s814"[39m) where T at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/givens.jl:63
[0m  copy([91m::Base.IdSet[39m) at idset.jl:15
[0m  ...

In [24]:
j = deepcopy(t)

Tower([Rod(), Rod(), Rod()])

In [26]:
j == t

false

In [6]:
UInt(8)

0x0000000000000008

In [20]:
a = reverse([Disc(each) for each in range(1, step=1,length=3)])

3-element Vector{Disc}:
 Disc(0x03)
 Disc(0x02)
 Disc(0x01)

In [18]:
reverse(a)

3-element Vector{Disc}:
 Disc(3)
 Disc(2)
 Disc(1)

In [21]:
Rod(a)

Rod(Disc[Disc(0x03), Disc(0x02), Disc(0x01)])

In [17]:
repeat("*", 3)

"***"

In [65]:
a=[1,2,3]

3-element Vector{Int64}:
 1
 2
 3

In [68]:
get(a, 3, '?')

3

In [85]:
for i in range(1, length=4)
    println(i)
end

1
2
3
4


In [147]:
hash(Data.Disc(7))

0x06f809fdd644aaa2

In [150]:
hash(deepcopy(Data.Disc(7)))

0x06f809fdd644aaa2

In [161]:
t = Data.Tower(4)

    ***                          
   *****                         
  *******                        
 *********                       


In [170]:
last(t.rods[1].discs)

***

In [371]:
tup = (1,2)

(1, 2)

In [374]:
v = [each for each in tup]

2-element Vector{Int64}:
 1
 2

In [375]:
push(tup, 4)

LoadError: UndefVarError: push not defined

In [462]:
t = Data.Tower(4)
println(hash(t))
println(hash(t.rods))
r = t.rods[1]
println(hash(r))
println(hash(r.discs))

3994254288365766802
3994254288365766802
11082675144903454004
11082675144903454004


In [463]:
t2 = Data.Tower(4)
println(hash(t2))
println(hash(t2.rods))
r2 = t2.rods[1]
println(hash(t2.rods[1]))
println(hash(t2.rods[1].discs))

3994254288365766802
3994254288365766802
11082675144903454004
11082675144903454004


In [464]:
t == t2

true

In [465]:
hash(t) == hash(t2)

true

In [466]:
r == r2

true

In [467]:
hash(r) == hash(r2)

true

In [434]:
r.discs == r2.discs

true

In [418]:
?==

search: == === !==



```
==(x, y)
```

Generic equality operator. Falls back to [`===`](@ref). Should be implemented for all types with a notion of equality, based on the abstract value that an instance represents. For example, all numeric types are compared by numeric value, ignoring type. Strings are compared as sequences of characters, ignoring encoding. For collections, `==` is generally called recursively on all contents, though other properties (like the shape for arrays) may also be taken into account.

This operator follows IEEE semantics for floating-point numbers: `0.0 == -0.0` and `NaN != NaN`.

The result is of type `Bool`, except when one of the operands is [`missing`](@ref), in which case `missing` is returned ([three-valued logic](https://en.wikipedia.org/wiki/Three-valued_logic)). For collections, `missing` is returned if at least one of the operands contains a `missing` value and all non-missing values are equal. Use [`isequal`](@ref) or [`===`](@ref) to always get a `Bool` result.

# Implementation

New numeric types should implement this function for two arguments of the new type, and handle comparison to other types via promotion rules where possible.

[`isequal`](@ref) falls back to `==`, so new methods of `==` will be used by the [`Dict`](@ref) type to compare keys. If your type will be used as a dictionary key, it should therefore also implement [`hash`](@ref).

---

```
==(x)
```

Create a function that compares its argument to `x` using [`==`](@ref), i.e. a function equivalent to `y -> y == x`.

The returned function is of type `Base.Fix2{typeof(==)}`, which can be used to implement specialized methods.

---

```
==(a::AbstractString, b::AbstractString) -> Bool
```

Test whether two strings are equal character by character (technically, Unicode code point by code point).

# Examples

```jldoctest
julia> "abc" == "abc"
true

julia> "abc" == "αβγ"
false
```


In [382]:
hash([1,2,3,4])

0xe5a77e407cb93a25

In [400]:
r.discs == r2.discs

true

In [398]:
[1,2,3] == [1,2,3]

true

In [399]:
[Data.Disc(i) for i in range(1,length=3)] == [Data.Disc(i) for i in range(1,length=3)] 

true

In [458]:
?hash

search: hash hasmethod haskey hasfield hasproperty skipchars Threads



```
hash(x[, h::UInt])
```

Compute an integer hash code such that `isequal(x,y)` implies `hash(x)==hash(y)`. The optional second argument `h` is a hash code to be mixed with the result.

New types should implement the 2-argument form, typically by calling the 2-argument `hash` method recursively in order to mix hashes of the contents with each other (and with `h`). Typically, any type that implements `hash` should also implement its own `==` (hence `isequal`) to guarantee the property mentioned above. Types supporting subtraction (operator `-`) should also implement [`widen`](@ref), which is required to hash values inside heterogeneous arrays.


LoadError: MethodError: no method matching hash(::Int64, ::Int64, ::Int64)
[0mClosest candidates are:
[0m  hash(::Int64, [91m::UInt64[39m) at hashing.jl:71
[0m  hash(::Real, [91m::UInt64[39m) at float.jl:489
[0m  hash(::Any) at hashing.jl:18
[0m  ...

In [556]:
using TowerOfHanoi

In [557]:
TowerOfHanoi.Tower(4)

    ***                          
   *****                         
  *******                        
 *********                       


In [555]:
using Revise

In [558]:
Tower(4)

LoadError: UndefVarError: Tower not defined

In [30]:
s = Set([1,2,3])
push!(s,4)
in(5, s)

false

In [31]:
in(4, s)

true

In [32]:
s = Set([Tower(4)])

Set{Tower} with 1 element:
      ***                          …

In [33]:
length(s)

1

In [34]:
push!(s, Tower(4))

Set{Tower} with 1 element:
      ***                          …

In [35]:
length(s)

1

In [36]:
in(Tower(4), s)

true

In [40]:
function example(func::Function, question)
    if question
        func(123)
    end
end

example(false) do a
    println(a)
end

example(true) do a
    println(a)
end

123


In [41]:
using DataStructures

In [43]:
data = collect(enumerate(["foo", "bar", "baz"]))

3-element Vector{Tuple{Int64, String}}:
 (1, "foo")
 (2, "bar")
 (3, "baz")

In [56]:
h1 = MutableBinaryHeap(Base.By(first),data) # Standard lexicographic ordering for tuples
first(h1)             # => (1, "foo")

(1, "foo")

In [53]:
h2 = MutableBinaryHeap(Base.By(last), data) # Order by 2nd element only
first(h2) 

(2, "bar")

In [50]:
h = MutableBinaryMinHeap{Int}()
h = MutableBinaryMaxHeap{Int}()      # create an empty mutable min/max heap

h = MutableBinaryMinHeap([1,4,3,2])
h = MutableBinaryMaxHeap([1,4,3,2]) 

MutableBinaryHeap(4, 2, 3, 1)

In [59]:
h = MutableBinaryMaxHeap([1,4,3,2]) 

MutableBinaryHeap(4, 2, 3, 1)

In [62]:
pop!(h)

4

In [63]:
pop!(h)

3

In [64]:
pop!(h)

2

In [74]:
data = [ 2, 4, 6, 8, 10, 12 ]
function heuristic(input)
    input % 3
end
h = MutableBinaryHeap(Base.By(heuristic), data)

MutableBinaryHeap(6, 10, 12, 8, 2, 4)

In [75]:
pop!(h)

6

In [76]:
pop!(h)

12

In [77]:
pop!(h)

10

In [78]:
pop!(h)

4

In [86]:
initial = Tower(4,1)
test_func = (state::Tower) -> heuristic(initial, state)

#15 (generic function with 1 method)

In [85]:
test_func(Tower(4,2))

LoadError: MethodError: no method matching heuristic(::Tower, ::Tower)
[0mClosest candidates are:
[0m  heuristic(::Any) at In[74]:2